In [1]:
!pip install streamlit huggingface_hub accelerate transformers torch langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 9.1 MB/s eta 0:00:00


In [8]:
%%writefile app.py
import transformers
import torch
from pprint import pprint
import streamlit as st
from langchain_core.messages import AIMessage, HumanMessage

def load_model_tokenizer(repository):
    model = transformers.AutoModelForCausalLM.from_pretrained(
        repository,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map = 'auto'
    )
    tokenizer = transformers.AutoTokenizer.from_pretrained(repository)
    return model, tokenizer


def get_response(text, model, tokenizer):
    system_message = "You are a world class fitness instructor and gym trainer, you will give proper exercise and diet plans if asked, always answer the use in detail. Always answer in bullet points.'"
    prompt = f"<|im_start|>system{system_message}<|im_end|><|im_start|>user\n{text}<|im_end|>\n<|im_start|>assistant:"
    input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=256)
    output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

    return output.split("<|im_end|>")[0]


st.set_page_config(page_title='Fitness Instructor', page_icon = "🏃‍♂️")

st.title("Fitness Instructor")


##Creating the chat_history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = [
        AIMessage(content="Hello I am hired as your Fitness Instructor. I will do my best to help you to the best of my Abilities.")
    ]


user_query = st.chat_input('Enter your Query here...')

if user_query is not None and user_query != "":
    model, tokenizer = load_model_tokenizer("AdityaLavaniya/TinyLlama-Fitness-Instructor")
    response = get_response(user_query, model, tokenizer)

    #Updating the chat_history:
    st.session_state.chat_history.append(HumanMessage(content = user_query ))
    st.session_state.chat_history.append(AIMessage(content = response))


    ##Displaying the chat_history in Application
    for message in st.session_state.chat_history:
        if isinstance(message, AIMessage):
            with st.chat_message("AI"):
                st.write(message.content)

        elif isinstance(message, HumanMessage):
            with st.chat_message("Human"):
                st.write(message.content)

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule yargs@17.1.1 ch


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.135.208.144:8501

npx: installed 22 in 1.987s
your url is: https://slimy-mammals-open.loca.lt
/root/.npm/_npx/3139/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:44689 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/3139/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (events.js:315:20)
    at emitErrorNT (internal/streams/destroy.js:106:8)
    at emitErrorCloseNT (internal/streams/destroy.js:74:3)
    at processTicksAndRejections (internal/process/task_queues.js:80:21)
